In [1]:
from pathlib import Path
from enum import Enum
import cv2
from cv2 import imwrite
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import preprocessing
from time import time
from scipy.optimize import curve_fit
%matplotlib inline
import scipy.stats
from util import parse_fn, is_bg
from scipy import stats
from scipy.stats import skewnorm
from scipy.special import erf
from scipy.special import voigt_profile
import re

# Input Here:  information of picture

In [2]:
HEADER_LEN = 152 
X_DIM = 480
Y_DIM = 640
X_left_boundry = 110
X_right_boundry = 260
Y_left_boundry = 305
Y_right_boundry = 355

## picture to blue light signal intensity

In [3]:
def read_raw_to_intensity(file):
    
    with open(file, "rb") as rawimage:
    
        bayer_image = np.fromfile(rawimage, np.dtype('u1'), offset = HEADER_LEN)
        fst_byte, scd_byte = bayer_image.reshape((bayer_image.shape[0]//2, 2)).astype(np.uint16).T
        raw_pixel = (fst_byte + (scd_byte << 8)).reshape((X_DIM, Y_DIM))

    return raw_pixel


def intensity_to_blue_16(intensity):
    intensity_uint16 = np.array(intensity, dtype = np.uint16)
    color = cv2.cvtColor(intensity_uint16, cv2.COLOR_BayerGRBG2RGB)
    blue = color[:,:,2]
    return blue


def get_blue(file):
    raw_pixel = read_raw_to_intensity(file)
    blue = intensity_to_blue_16(raw_pixel)
    return blue.astype(np.float64)

## get the bright region

In [4]:
def get_bright(pure_blue):
    data = pure_blue[X_left_boundry:X_right_boundry, Y_left_boundry:Y_right_boundry]
    return data

## get blue signal of interested region : get_data(file)

In [5]:
# 拿到我要的範圍的blue
def get_data(file):
    blue = get_blue(file)
    bright = get_bright(blue)
    return bright

## get final fitting peak dR / R (one value) of a frame

In [6]:
def peak_dRoverR(file, b_file, func, maxs_func):
    dRoverR = get_dRoverR(file, b_file)
    maxs = get_maxs(dRoverR, func)
    peak = get_max(maxs, maxs_func)
    return peak

In [7]:
def get_peak_dRoverR(file, back_intensity, func, maxs_func):
    dRoverR = get_dR_over_R(file, back_intensity)
    maxs = get_maxs(dRoverR, func)
    peak = get_max(maxs, maxs_func)
    return peak

## get dR / R of every point of interested in a frame

In [8]:
def get_dRoverR(file, b_file):
    blue = get_data(file)
    b_blue = get_data(b_file)
    dRoverR = (blue - b_blue) / b_blue
    return dRoverR

In [9]:
def get_dR_over_R(file, back_intensity):
    blue = get_data(file)
    b_blue = get_bright(back_intensity)
    dRoverR = (blue - b_blue) / b_blue
    return dRoverR